## KNN Model

#### Librerias a utilizar

In [721]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import os
from flask_sqlalchemy import SQLAlchemy
import sqlalchemy
from sqlalchemy import *
import psycopg2
from sqlalchemy.sql import *
import json
from psycopg2 import extras
import warnings
warnings.filterwarnings("ignore")
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


#### Paso 1. Lectura del conjunto de datos

In [722]:
data_m=pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv',encoding='utf-8')
data_c=pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv', encoding='utf-8')

In [723]:
print(data_m.shape, data_c.shape)

(4803, 20) (4803, 4)


In [724]:
data_m.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [725]:
data_m.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [726]:
data_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [727]:
data_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [728]:
dups=data_c['title'].duplicated().sum()
print(dups)

registros_dup=data_c[data_c['title'].duplicated(keep=False)]
registros_dup

data_m.drop_duplicates(subset=['title'], inplace=True)	
data_c.drop_duplicates(subset=['title'], inplace=True)	

3


In [729]:
data_m.drop_duplicates(subset=['title'], inplace=True)	

In [730]:
data_m['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

##### Procesamiento de la data previa carga en Postgret SQL

- Eliminando espacios en blanco al principio y final de las columnas tipo string (simultaneo se transforman en minusculas) y las numericas.

- Se crea funcion clean string para diferenciar de forma adecuada los tipo de data existentes

In [731]:
def clean_string(x):
    if isinstance(x, str):  # solo aplicar a strings
        return x.strip().lower()
    elif isinstance(x, dict) or isinstance(x, list):  # si es un objeto JSON, dejarlo intacto
        return x
    else:
        return x  # si es otro tipo de dato, dejarlo intacto

for colm in data_m.columns:
    if data_m[colm].dtype.kind in ['i', 'f']:  # solo aplicar a columnas de tipo int o float
        data_m[colm] = data_m[colm].astype(str).str.strip().astype(data_m[colm].dtype)
    else:
        data_m[colm] = data_m[colm].apply(clean_string)

for colc in data_c.columns:
    if data_c[colc].dtype.kind in ['i', 'f']:  # solo aplicar a columnas de tipo int o float
        data_c[colc] = data_c[colc].astype(str).str.strip().astype(data_c[colc].dtype)
    else:
        data_c[colc] = data_c[colc].apply(clean_string)

In [732]:
data_m.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4800 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4800 non-null   int64  
 1   genres                4800 non-null   object 
 2   homepage              1711 non-null   object 
 3   id                    4800 non-null   int64  
 4   keywords              4800 non-null   object 
 5   original_language     4800 non-null   object 
 6   original_title        4800 non-null   object 
 7   overview              4797 non-null   object 
 8   popularity            4800 non-null   float64
 9   production_companies  4800 non-null   object 
 10  production_countries  4800 non-null   object 
 11  release_date          4799 non-null   object 
 12  revenue               4800 non-null   int64  
 13  runtime               4798 non-null   float64
 14  spoken_languages      4800 non-null   object 
 15  status                4800

In [733]:
data_c.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4800 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4800 non-null   int64 
 1   title     4800 non-null   object
 2   cast      4800 non-null   object
 3   crew      4800 non-null   object
dtypes: int64(1), object(3)
memory usage: 187.5+ KB


#### Paso 2: Creación de una base de datos

In [734]:
# Credenciales Base de Datos

user = 'postgres'       #0
password = 'america'    #1
host = 'localhost'      #2
port = 5432             #3
database = 'postgres'   #4

# Conectandome a mi Base de Datos en PostgreSQL: 

conn = psycopg2.connect(database=database, user=user, host=host, password=password, port=port)
cur = conn.cursor()

# Verificando conexion con Postgres SQL
if isinstance(conn, psycopg2.extensions.connection):
    print("Conectado correctamente a la base de datos")



Conectado correctamente a la base de datos


In [735]:

# Función para convertir DataFrame a lista de tuplas
def df_to_tuplas(df):
    return [tuple(x) for x in df.to_numpy()]

# Insertar datos en la tabla Peliculas
def insert_data(df, table_name):
    tuplas = df_to_tuplas(df)
    cols = ','.join([f'"{col}"' for col in df.columns])
    query  = f"INSERT INTO {table_name}({cols}) VALUES %s"
    psycopg2.extras.execute_values(cur, query, tuplas)


In [736]:
# Crear tablas e insertar datos
conn = psycopg2.connect(database=database, user=user, host=host, password=password, port=port)
cur = conn.cursor()
try:
    cur.execute("DROP TABLE IF EXISTS Peliculas")
    cur.execute("""
    CREATE TABLE Peliculas (
        budget bigint, 
        genres text, 
        homepage text, 
        id bigint, 
        keywords text, 
        original_language character varying(10), 
        original_title text, 
        overview text, 
        popularity double precision, 
        production_companies text, 
        production_countries text, 
        release_date character varying(15), 
        revenue bigint, 
        runtime double precision, 
        spoken_languages text, 
        status text, 
        tagline text, 
        title text, 
        vote_average double precision, 
        vote_count bigint,
        PRIMARY KEY(title))
""")
    cur.execute("DROP TABLE IF EXISTS Creditos")
    cur.execute("""CREATE TABLE Creditos (movie_id integer, title text, crew text, 
                "cast" text, PRIMARY KEY(title))""")         
    

    insert_data(data_m, 'Peliculas')
    insert_data(data_c, 'Creditos')

    conn.commit()
    print("Tablas creadas e insertadas exitosamente.")
except UnicodeDecodeError as e:
    print(f"Error de codificación: {e}")
#except Exception as e:
    print(f"Ha ocurrido un error: {e}")
finally:
    cur.close()
    conn.close()



Tablas creadas e insertadas exitosamente.


In [737]:
conn = psycopg2.connect(database=database, user=user, host=host, password=password, port=port)
cur = conn.cursor()

# Ejecutar una consulta SQL para seleccionar todos los datos de la tabla
cur.execute("SELECT * FROM peliculas_creditos")

# Obtener los resultados de la consulta
rows = cur.fetchall()

# Cerrar el cursor
cur.close()
# Cerrar la conexión
conn.close()

# Convertir los resultados en un DataFrame de Pandas
data_f = pd.DataFrame(rows, columns=[column[0] for column in cur.description])

data_f.shape

(4800, 23)

In [738]:
cols=['movie_id','title','overview','genres','keywords','cast','crew']
data_peliculas=data_f[cols]
data_peliculas.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,avatar,"in the 22nd century, a paraplegic marine is di...","[{""id"": 28, ""name"": ""action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""jake sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,pirates of the caribbean: at world's end,"captain barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""captain jack spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,spectre,a cryptic message from bond’s past sends him o...,"[{""id"": 28, ""name"": ""action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""james bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,the dark knight rises,following the death of district attorney harve...,"[{""id"": 28, ""name"": ""action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""bruce wayne / ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,john carter,"john carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""john carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


#### Paso 3. Procesando columnas del dataframe consolidado
##### Seleccionando el atributo `name` y reemplazando las columnas `genres` y `keywords`. Para la columna `cast` seleccionando los tres primeros nombres. Seleccionando nombre del director para la columna `crew`. Generando lista con resumen de las peliculas.

In [739]:
data_peliculas['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "editing", "gender": 0, "id": 1721, "job": "editor", "name": "stephen e. rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "art", "gender": 2, "id": 496, "job": "production design", "name": "rick carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "sound", "gender": 0, "id": 900, "job": "sound designer", "name": "christopher boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "sound", "gender": 0, "id": 900, "job": "supervising sound editor", "name": "christopher boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "production", "gender": 1, "id": 1262, "job": "casting", "name": "mali finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "sound", "gender": 2, "id": 1729, "job": "original music composer", "name": "james horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "directing", "gender": 2, "id": 2710, "job": "director", "name": "james cameron"},

In [740]:
def process_genres(genres):
    datos = json.loads(genres)
    names = [dato["name"] for dato in datos]
    return names

data_peliculas['genres_m'] = data_peliculas['genres'].apply(process_genres)
data_peliculas.drop('genres', inplace=True, axis=1)
data_peliculas.rename(columns={'genres_m': 'genres'}, inplace=True)

data_peliculas['keyw_m'] = data_peliculas['keywords'].apply(process_genres)
data_peliculas.drop('keywords', inplace=True, axis=1)
data_peliculas.rename(columns={'keyw_m': 'keywords'}, inplace=True)

def process_genres3(cast):
    datos = json.loads(cast)
    names = [dato["name"] for dato in datos]
    return names[:3] 
data_peliculas['cast_3n'] = data_peliculas['cast'].apply(process_genres3)
data_peliculas.drop('cast', inplace=True, axis=1)
data_peliculas.rename(columns={'cast_3n': 'cast'}, inplace=True)

def process_director(crew):
    datos = json.loads(crew)
    director_name = [dato["name"] for dato in datos if dato["job"]=="director"]
    return director_name
data_peliculas['crew_d'] = data_peliculas['crew'].apply(process_director)
data_peliculas.drop('crew', inplace=True, axis=1)
data_peliculas.rename(columns={'crew_d': 'crew'}, inplace=True)

resumen=data_peliculas.overview.to_list()

In [741]:
data_peliculas.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,avatar,"in the 22nd century, a paraplegic marine is di...","[action, adventure, fantasy, science fiction]","[culture clash, future, space war, space colon...","[sam worthington, zoe saldana, sigourney weaver]",[james cameron]
1,285,pirates of the caribbean: at world's end,"captain barbossa, long believed to be dead, ha...","[adventure, fantasy, action]","[ocean, drug abuse, exotic island, east india ...","[johnny depp, orlando bloom, keira knightley]",[gore verbinski]
2,206647,spectre,a cryptic message from bond’s past sends him o...,"[action, adventure, crime]","[spy, based on novel, secret agent, sequel, mi...","[daniel craig, christoph waltz, léa seydoux]",[sam mendes]
3,49026,the dark knight rises,following the death of district attorney harve...,"[action, crime, drama, thriller]","[dc comics, crime fighter, terrorist, secret i...","[christian bale, michael caine, gary oldman]",[christopher nolan]
4,49529,john carter,"john carter is a war-weary, former military ca...","[action, adventure, science fiction]","[based on novel, mars, medallion, space travel...","[taylor kitsch, lynn collins, samantha morton]",[andrew stanton]


##### Eliminando los espacios en blanco en los elementos de las listas en las columnas `genres`, `cast`, `crew` y `keywords`

In [742]:
def eliminar_espacios(valor):
    return [str(elemento_lista).replace(" ", "") for elemento_lista in valor]

columnas_a_modificar = ['genres', 'cast', 'crew', 'keywords']

data_peliculas[columnas_a_modificar] = data_peliculas[columnas_a_modificar].applymap(eliminar_espacios)


In [743]:
data_peliculas.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,avatar,"in the 22nd century, a paraplegic marine is di...","[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...","[samworthington, zoesaldana, sigourneyweaver]",[jamescameron]
1,285,pirates of the caribbean: at world's end,"captain barbossa, long believed to be dead, ha...","[adventure, fantasy, action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[johnnydepp, orlandobloom, keiraknightley]",[goreverbinski]
2,206647,spectre,a cryptic message from bond’s past sends him o...,"[action, adventure, crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[danielcraig, christophwaltz, léaseydoux]",[sammendes]
3,49026,the dark knight rises,following the death of district attorney harve...,"[action, crime, drama, thriller]","[dccomics, crimefighter, terrorist, secretiden...","[christianbale, michaelcaine, garyoldman]",[christophernolan]
4,49529,john carter,"john carter is a war-weary, former military ca...","[action, adventure, sciencefiction]","[basedonnovel, mars, medallion, spacetravel, p...","[taylorkitsch, lynncollins, samanthamorton]",[andrewstanton]


In [744]:
resumen[0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [745]:
data_peliculas.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,avatar,"in the 22nd century, a paraplegic marine is di...","[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...","[samworthington, zoesaldana, sigourneyweaver]",[jamescameron]
1,285,pirates of the caribbean: at world's end,"captain barbossa, long believed to be dead, ha...","[adventure, fantasy, action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[johnnydepp, orlandobloom, keiraknightley]",[goreverbinski]
2,206647,spectre,a cryptic message from bond’s past sends him o...,"[action, adventure, crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[danielcraig, christophwaltz, léaseydoux]",[sammendes]
3,49026,the dark knight rises,following the death of district attorney harve...,"[action, crime, drama, thriller]","[dccomics, crimefighter, terrorist, secretiden...","[christianbale, michaelcaine, garyoldman]",[christophernolan]
4,49529,john carter,"john carter is a war-weary, former military ca...","[action, adventure, sciencefiction]","[basedonnovel, mars, medallion, spacetravel, p...","[taylorkitsch, lynncollins, samanthamorton]",[andrewstanton]


##### Unificando las columnas: `genres`, `keywords`, `cast` y `crew`

In [746]:
data_peliculas['unificado']=data_peliculas['genres']+data_peliculas['keywords']+data_peliculas['cast']+data_peliculas['crew']
data_peliculas['unificado'].head()
def eliminar_comas(valor):
    return [str(elemento_lista).replace(",","") for elemento_lista in valor]

data_peliculas['unificado'] = data_peliculas['unificado'].apply(eliminar_comas)
data_peliculas['unificado'] = data_peliculas['unificado'].apply(lambda x: ' '.join(x))
data_peliculas['unificado'].head()

0    action adventure fantasy sciencefiction cultur...
1    adventure fantasy action ocean drugabuse exoti...
2    action adventure crime spy basedonnovel secret...
3    action crime drama thriller dccomics crimefigh...
4    action adventure sciencefiction basedonnovel m...
Name: unificado, dtype: object

In [747]:
data_knn=pd.DataFrame(columns=['tags','title'])
data_knn['tags'] = data_peliculas['overview'].str.cat(data_peliculas['unificado'], sep=' ')
data_knn['title']=data_peliculas['title'] 

In [748]:
data_knn.head()

,tags,title
0,"in the 22nd century, a paraplegic marine is di...",avatar
1,"captain barbossa, long believed to be dead, ha...",pirates of the caribbean: at world's end
2,a cryptic message from bond’s past sends him o...,spectre
3,following the death of district attorney harve...,the dark knight rises
4,"john carter is a war-weary, former military ca...",john carter


In [749]:
data_knn.tags[0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

### Paso 4: Construccion del KNN

#### Vectorizacion de la Variable tags

In [750]:
X=pd.Series(data_knn['tags'])
X.head()

0    in the 22nd century, a paraplegic marine is di...
1    captain barbossa, long believed to be dead, ha...
2    a cryptic message from bond’s past sends him o...
3    following the death of district attorney harve...
4    john carter is a war-weary, former military ca...
Name: tags, dtype: object

In [751]:
def normalize_text(text):
    return unicodedata.normalize('NFKD', text)

In [752]:
vec_model = CountVectorizer(stop_words = "english", preprocessor=normalize_text)
Xvec = vec_model.fit_transform(X).toarray()

#### Uso Data Vectorizada
- En este caso no tenemos una variable a predecir, nuestro código será soportado en la función cosine_similarity aplicada sobre la columna **tags** vectorizada, para hacer recomendaciones de 5 peliculas similares en función a la selección de una pelicula del DATASET. Si la pelicula no esta en el DATASET, el código arrojará un error de indice y debemos intentar con otra.

In [753]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(Xvec)

In [754]:
def recommend(movie):
    try:
        movie_index = data_knn[data_knn["title"] == movie.lower()].index[0]
        distances = similarity[movie_index]
        movie_list = sorted(list(enumerate(distances)), reverse = True , key = lambda x: x[1])[1:6]
        for i in movie_list:
            print(data_knn.iloc[i[0]].title)  
    except IndexError:
        print("Error: La pelicula no esta en el DATASET. Intente con otra !!!")  

In [755]:
recommend("gladiator")

hercules
pompeii
the last legion
we were soldiers
underworld: rise of the lycans
